In [1]:
%matplotlib inline
import seaborn as sns; 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import gridspec
import scipy
style.use('ggplot')
import datetime
from io import StringIO
import re
import os
import numpy as np
from datetime import datetime

# Images build date

In [2]:
build_date = pd.concat([pd.read_csv('../../data/prepared_data/'+file+'.csv') 
                        for file in ['build_date_official','build_date_community']])
build_date.shape

(141068, 2)

In [3]:
# add "library/" to official images
def add_library(s):
    if len(s.split(':'))==2:
        return 'library:'+s
    return s
build_date['image'] = build_date['image'].apply(lambda x: add_library(x))

# Images popularity

In [4]:
images = pd.concat([pd.read_csv('../../data/images_tags/'+file+'.zip', usecols=['slug','popularity']) 
                        for file in ['official_images','community_images']])
images.shape

(2188015, 2)

In [5]:
# add "library/" to official images
def add_library(s):
    if len(s.split('/'))==1:
        return 'library/'+s
    return s
images['slug'] = images['slug'].apply(lambda x: add_library(x))

In [6]:
images.popularity.describe().apply(lambda x: format(x, 'f'))

count       2188015.000000
mean          45035.516850
std         5129680.538336
min               0.000000
25%               5.000000
50%              21.000000
75%              80.000000
max      2146530870.000000
Name: popularity, dtype: object

In [7]:
# get slug from inspected images
def split_slug(s):
    s = s.split(':')
    return s[0]+'/'+s[1]
build_date['slug'] = build_date['image'].apply(lambda x: split_slug(x))

In [8]:
images['popularity'] = images['popularity'].apply(int)

In [9]:
images.popularity.describe().apply(lambda x: format(x, 'f'))

count       2188015.000000
mean          45035.516850
std         5129680.538336
min               0.000000
25%               5.000000
50%              21.000000
75%              80.000000
max      2146530870.000000
Name: popularity, dtype: object

In [13]:
images_debian = pd.read_csv('../../data/images_tags/community_debian_images.csv')
images_debian.shape

(923969, 2)

In [14]:
images_debian['slug'] = images_debian['image'].apply(lambda x: x.split(':')[0]+'/'+x.split(':')[1])

In [15]:
images_debian.head()

,index,image,slug
0,0,010309:ams-go:0.1.0,010309/ams-go
1,1,010309:bia-excass:0.1.0,010309/bia-excass
2,2,010309:bia-mailsender:0.1.0,010309/bia-mailsender
3,3,010309:bia-query:0.1.0,010309/bia-query
4,4,010309:biaasteriskcdr:0.1.0,010309/biaasteriskcdr


In [16]:
# all Debian images
mask = images.slug.isin(images_debian.slug)
images2 = images[mask]

In [17]:
images.quantile(0.955)

popularity    1880.0
Name: 0.955, dtype: float64

In [18]:
images2.quantile(0.947)

popularity    1983.824
Name: 0.947, dtype: float64

In [19]:
images2.popularity.describe().apply(lambda x: format(x, 'f'))

count        400193.000000
mean          28219.973598
std         3940371.038902
min               0.000000
25%               6.000000
50%              17.000000
75%              70.000000
max      2047714388.000000
Name: popularity, dtype: object

In [20]:
images.popularity.describe().apply(lambda x: format(x, 'f'))

count       2188015.000000
mean          45035.516850
std         5129680.538336
min               0.000000
25%               5.000000
50%              21.000000
75%              80.000000
max      2146530870.000000
Name: popularity, dtype: object

In [22]:
# only those we used
mask = images.slug.isin(build_date.slug)
images3 = images[mask]

In [29]:
images3[images3['slug'].str.startswith('library/')].sort_values('popularity').tail(10)

,slug,popularity
349,library/golang,511220212
381,library/mariadb,553694529
379,library/memcached,738063984
289,library/mysql,846275977
321,library/node,869742810
288,library/postgres,1016798085
378,library/httpd,1135282151
320,library/redis,1590340952
262,library/mongo,1760010241
261,library/nginx,1804165577


In [30]:
images3[~images3['slug'].str.startswith('library/')].sort_values('popularity').tail(10)

,slug,popularity
163031,appsvc/node,74751359
370827,circleci/postgres,77022944
306409,buoyantio/linkerd,84358536
370819,circleci/node,89457273
2072940,whenlambomoon/secnodetracker,91997588
942624,jenkins/jenkins,99522070
57610,aequitas/http-api-resource,130646930
1873500,sysdig/agent,157178862
1416562,nulldriver/maven-resource,171307389
1506435,pivotalcf/pivnet-resource,2047714388


In [72]:
build_date = (build_date.merge(images,
                              left_on = 'slug',
                               right_on = 'slug',
                               how='left'))
build_date.shape

(141068, 4)

# Images update date

In [105]:
tags = pd.concat([pd.read_csv('../../data/images_tags/'+file+'.csv', usecols=['slug','tag','last_updated']) 
                        for file in ['official_tags','community_tags']])
tags.shape

(5870325, 3)

In [106]:
def add_library(s):
    if len(s.split('/'))==1:
        return 'library/'+s
    return s
tags['slug'] = tags['slug'].apply(lambda x: add_library(x))
tags['image'] = tags['slug']+':'+tags['tag']

def split_slug(s):
    s = s.split(':')
    return s[0]+'/'+s[1]

tags['image'] = tags['image'].apply(lambda x: str(x).replace('/',':'))
tags.shape

(5870325, 4)

In [107]:
mask = tags.slug.isin(build_date.slug)
tags = tags[mask]
tags.shape

(270591, 4)

In [108]:
mask = tags.image.isin(build_date.image)
tags = tags[mask]
tags = tags[['image','last_updated']]
tags.shape

(141068, 4)

In [111]:
build_date = (build_date.merge(tags,
                              left_on = 'image',
                               right_on = 'image',
                               how='left'))
build_date.shape

(141068, 5)

# Load image packages

In [2]:
packages = pd.concat([pd.read_csv('../../data/prepared_data/list_pkgs_rels/'+file) 
                        for file in ['officila_packs.csv','community_packs.csv']])
packages.rename(columns={'name':'image'}, inplace=True)
packages['package'] = packages['package'].apply(lambda x: str(x).replace(':amd64',''))
packages.shape

(47094186, 3)

In [3]:
releases = pd.concat([pd.read_csv('../../data/prepared_data/list_pkgs_rels/'+file) 
                        for file in ['officila_releases.csv','community_releases.csv']])
releases.rename(columns={'name':'image', 'release':'os'}, inplace=True)

releases.shape

(141077, 2)

In [4]:
def get_os(s):
    try:
        if float(s)>=8:
            if float(s)<9:
                return 'jessie'
            elif float(s)<10:
                return 'stretch'
            else:
                return 'buster'
        else:
            return 'other'
    except:
        if 'stretch' in s:
            return 'stretch'
        elif 'buster' in s:
            return 'buster'
#         elif 'bullseye' in s:
#             return 'bullseye'
#         elif 'parrot' in s:
#             return 'parrot'
        else:
            return 'other'
        
releases['release'] = releases['os'].apply(lambda x: get_os(x))
releases.drop('os', axis=1, inplace=True)

In [5]:
releases.groupby('release').count()

,image
release,
buster,3855
jessie,55611
other,551
stretch,81060


In [6]:
packages = (packages.merge(releases,
                          left_on = 'image',
                          right_on = 'image',
                          how = 'left')
           )
packages.shape

(47094186, 4)

In [7]:
packages.dropna().shape

(47088868, 4)

In [8]:
packages.fillna('undefined', inplace=True)
packages = packages.query('release != "other"').copy()

In [9]:
packages.shape

(46943669, 4)

In [10]:
packages[['package','version','release']].drop_duplicates().groupby('release').count()

,package,version
release,,
buster,18111,18111
jessie,25510,25510
stretch,30238,30238
undefined,1969,1969


In [11]:
packages[['package','version']].drop_duplicates().shape

(61807, 2)

# Load packages from Debian

In [3]:
debian_packages = pd.read_csv('../../data/prepared_data/debian_packages.csv')
debian_packages.shape

(804645, 5)

In [4]:
debian_packages.query('version')

,package,version,archive,date,source
0,0ad,0.0.13-1,jessie,20130506,0ad
1,0ad,0.0.13-2,jessie,20130604,0ad
2,0ad,0.0.13-2+b1,jessie,20130702,0ad (0.0.13-2)
3,0ad,0.0.13-2+b2,jessie,20130901,0ad (0.0.13-2)
4,0ad,0.0.14-2,jessie,20130917,0ad


In [13]:
mask = debian_packages.package.isin(packages.package)
debian_packages = debian_packages[mask]

mask = debian_packages.version.isin(packages.version)
debian_packages = debian_packages[mask]

debian_packages.shape

(78478, 5)

In [14]:
packages.shape

(46943669, 4)

In [15]:
packages = (packages
            .set_index(['package','version'])
            .merge(debian_packages
                   .set_index(['package','version']),
                   left_index = True,
                   right_index = True,
                   how = 'left')
           )
packages.reset_index(inplace=True)
packages.shape

(46943669, 7)

In [16]:
packages.fillna('unknown', inplace=True)

In [17]:
packages.groupby('archive').count()*100/len(packages)

,package,version,image,release,date,source
archive,,,,,,
bullseye,0.005153,0.005153,0.005153,0.005153,0.005153,0.005153
buster,2.472900,2.472900,2.472900,2.472900,2.472900,2.472900
buster-security,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032
jessie,31.035026,31.035026,31.035026,31.035026,31.035026,31.035026
jessie-security,10.101675,10.101675,10.101675,10.101675,10.101675,10.101675
stretch,44.083527,44.083527,44.083527,44.083527,44.083527,44.083527
stretch-security,11.480730,11.480730,11.480730,11.480730,11.480730,11.480730
unknown,0.820956,0.820956,0.820956,0.820956,0.820956,0.820956


In [18]:
packages = packages.query('archive != "unknown"')
packages.head(2)

,package,version,image,release,archive,date,source
0,0ad,0.0.21-2,roadmunk:largecontainer:latest,stretch,stretch,2.01611e+07,0ad
1,0ad-data,0.0.21-1,roadmunk:largecontainer:latest,stretch,stretch,2.01611e+07,0ad-data


In [19]:
packages.shape

(46558282, 7)

In [20]:
# add "library/" to official images
def add_library(s):
    if len(s.split(':'))==2:
        return 'library:'+s
    return s
packages['image'] = packages['image'].apply(lambda x: add_library(x))

### Now we merge the build date with the packages DF

In [4]:
packages = (packages
            .merge(build_date,
                   left_on = "image",
                   right_on = "image",
                   how = 'left')
           )
packages.shape

(46558282, 9)

In [5]:
def typo(s):
    if s.startswith('library:'):
        return 'o'
    else:
        return 'c'
    
packages['type'] = packages['image'].apply(lambda x: typo(x))

In [6]:
packages.dropna().shape

(46558282, 10)

In [7]:
packages['last_updated'] = packages['last_updated'].apply(lambda x: x.split('T')[0])

In [9]:
packages['date'] = packages['date'].apply(str)

In [ ]:
packages['date'] = packages['date'].apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:8])

In [8]:
packages.to_csv('../../data/for_analysis/installed_packages.csv', index=False)